In [1]:
import sys
#sys.path.append('/usr/lib/python3/dist-packages')
# print(sys.path)
import TimeTagger
from TimeTagger import *
from TimeTagger import createTimeTagger, Counter, Countrate, Histogram, LOW, HIGH, freeTimeTagger, scanTimeTagger, ConstantFractionDiscriminator

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# import oskar
# scripts = oskar.Scripts()
import numpy as np
from tqdm import tqdm
import numpy
import matplotlib.pyplot as plot
import sys
import pickle as pkl
# import pyvisa as visa
import visa
import datetime
from scipy.optimize import curve_fit
from collections import defaultdict
from scipy.stats import kurtosis, skew
import pandas as pd
import xarray as xr
from collections import OrderedDict
from pathlib import Path
import fnmatch
from time import sleep

In [5]:
print (scanTimeTagger())
# create a timetagger instance
tagger = createTimeTagger()
# print (scanTimeTagger())
tagger.reset()
trigger = 0.5
tagger.setTriggerLevel(1, trigger)
tagger.setDeadtime(1, 100000)
# Ch2_CFD = ConstantFractionDiscriminator(tagger,[2], search_window = 10000)
# CFD_channels = Ch2_CFD.getChannels()

('2138000XGZ',)


ERR: Failed to claim USB interface: Device or resource busy (error 16)


RuntimeError: No Time Tagger attached or already in use. Got the USB error 'Failed' in 'OpenBySerial' in line 153.

Quick and dirty idea: run the single block twice for each experiment; in artiq, have another channel that has a constant delay after the triggering pulse. Sleep for the same amount of time, the number of experiment cycles will be the number of this pulses.

Another idea: after running the quick and dirty version of the code (compare it with artiq's input, roi counting and histogram counting). If we gain confidence for artiq, then we could use only artiq to take the measurement, which will be more simple.

# Double histogram for electron signal and sanity check cycle pulse

In [6]:
variable = []
n_cycle = []
roi = []
prob = []

In [7]:
wait_times = np.logspace(3, 6, 10) * 1E-6 # in s
wait_times_random = wait_times
print (wait_times)
np.random.shuffle(wait_times_random)
print (wait_times_random)

t_wait = 10E-6 # in us
wait_times_random = [t_wait]
cycle_duration = 112.2E-6 + t_wait # count this in artiq, double check with a awg, t_load (us) + t_wait (us) + 2.2 + 10
exp_cycles = 5E4  # how many experiment repetitions
########## for calibrated ttl12 #####################
time_window_calibration = 1000 # total time window width (after the triggering) in ns
t_start_calibration = 100 # start time of the roi after the triggering in ns
t_stop_calibration = 300 # stop time of the roi after the triggering in ns
########## for electron signal ######################
time_window = 1000 # total time window width (after the triggering) in ns
t_start = 450 # start time of the roi after the triggering in ns
t_stop = 650 # stop time of the roi after the triggering in ns
bwidth = 500 # binwidth of time tagger histogram in ps
date = datetime.datetime.today().strftime("%Y%m%d")


for wait_time in tqdm(wait_times_random):

#     cycle_duration,wait_time_corr = rigol.run(pulse_width_ao, pulse_delay_ao, pulse_width_ej, pulse_delay_ej, wait_time)
    filename = date + "_Hist_" + str("{:n}".format(wait_time/1E-6))
    '''
    filename = date + '_lifetime_U2_' + str("{:.2f}".format(U2))+'_U1_' + str("{:.2f}".format(U1))  +'_Ex' + str("{:.2f}".format(Ex)) +'_Ey' + str("{:.2f}".format(Ey))+'_Ez' + str("{:.2f}".format(Ez)) + '_Prf' + str("{:.2f}".format(Prf)) +'dBm_Tload' + str("{:n}".format(pulse_width_ao/1E-6)) + 'us_Tej' \
    + str("{:n}".format(pulse_width_ej/1E-9)) + 'ns_wait' + str("{:n}".format(wait_time_corr/1E-6)) + 'us_reps'+ \
    str("{:.1e}".format(exp_cycles)) +'_trigger' + str("{:.1f}".format(trigger))+ 'CFD_off_+300V.txt'
    '''
    print (filename)
    ###############################
    # set up time tagger histogram and take data
    n_bin = int(time_window*(1000/bwidth)) # total time window 500 ns
    histo = Histogram(tagger, click_channel = 2, start_channel = 1, binwidth = bwidth, n_bins = n_bin)
    histo_calibration = Histogram(tagger, click_channel = 3, start_channel = 1, binwidth = bwidth, n_bins = n_bin)
#     histo = Histogram(tagger, click_channel = CFD_channels[0], start_channel = 1, binwidth = bwidth, n_bins = n_bin)
    tagger.sync()
    histo.clear()
    histo_calibration.clear()
    sleep(cycle_duration * exp_cycles)
    data = histo.getData()
    data_calibration = histo_calibration.getData()
    ###############################
    # save data for the signal
    roi_start = int(t_start * 1000/bwidth)  # start time of the roi
    roi_stop = int(t_stop *1000/bwidth)   # stop time of the roi
    roi_start_calibration = int(t_start_calibration * 1000/bwidth)  # start time of the roi
    roi_stop_calibration = int(t_stop_calibration *1000/bwidth)   # stop time of the roi
    x_data = np.arange(0, n_bin) * bwidth/1000 # x-axis, time in ns
    file_data = np.array([x_data, data])
    file_data = file_data.T
    np.savetxt(filename, file_data, delimiter=',')
    roi_count = np.sum(data[roi_start:roi_stop])
    cycle_numbers = np.sum(data_calibration[roi_start_calibration:roi_stop_calibration])
    print (wait_time*1E3, roi_count, cycle_numbers, roi_count/cycle_numbers)
    variable = np.append(variable, wait_time)
    n_cycle = np.append(n_cycle, cycle_numbers)
    roi = np.append(roi, roi_count)
    prob = np.append(prob, roi_count/cycle_numbers)



[0.001      0.00215443 0.00464159 0.01       0.02154435 0.04641589
 0.1        0.21544347 0.46415888 1.        ]
[0.02154435 0.04641589 0.46415888 0.1        0.21544347 0.001
 0.01       0.00464159 0.00215443 1.        ]


  0%|                                                     | 0/1 [00:00<?, ?it/s]

20221116_Hist_10


NameError: name 'tagger' is not defined

In [20]:
print(variable)
print(n_cycle)
print(roi)
print(prob)

[1.e-05 1.e-04 1.e-03]
[41008. 54259. 86536.]
[1275.  788.   18.]
[0.03109149 0.01452294 0.00020801]


In [21]:
ROI_data = np.array([variable,roi,n_cycle])
ROI_data = ROI_data.T 
'''
    filename_roi = date + '_lifetime_ROI_U2_' + str("{:.2f}".format(U2))+'_U1_' + str("{:.2f}".format(U1)) +'_Ex' + str("{:.2f}".format(Ex)) +'_Ey' + str("{:.2f}".format(Ey)) +'_Ez' + str("{:.2f}".format(Ez))+'_Prf' + str("{:.2f}".format(Prf)) +'dBm_Tej' \
    + str("{:n}".format(pulse_width_ej/1E-9)) + 'ns_wait' + str("{:n}".format(wait_time_corr/1E-6)) + 'us_Tload' + str("{:n}".format(pulse_width_ao/1E-6)) + 'us_' + 'reps'+ \
    str("{:.1e}".format(exp_cycles)) + '_trigger' + str("{:.1f}".format(trigger))+ 'CFD_off_+300V.txt'
'''
filename_roi = date + "_ROI_lifetime_1"
np.savetxt(filename_roi,ROI_data,delimiter=',')

# Single histogram for electron signal

In [ ]:
wait_times = np.logspace(3, 6, 10) * 1E-6 # in s
wait_times_random = wait_times
print (wait_times)
np.random.shuffle(wait_times_random)
print (wait_times_random)


wait_times_random = [100E-6]
cycle_duration = 117.2E-6 # count this in artiq, double check with a awg, t_load (us) + t_wait (us) + 2.2 + 10
exp_cycles = 5E3  # how many experiment repetitions
########## for electron signal ######################
time_window = 500 # total time window width (after the triggering) in ns
t_start = 80 # start time of the roi after the triggering in ns
t_stop = 130 # stop time of the roi after the triggering in ns
bwidth = 500 # binwidth of time tagger histogram in ps
date = datetime.datetime.today().strftime("%Y%m%d")
variable = []
roi = []


for wait_time in tqdm(wait_times_random):

#     cycle_duration,wait_time_corr = rigol.run(pulse_width_ao, pulse_delay_ao, pulse_width_ej, pulse_delay_ej, wait_time)
    filename = date + "_Hist_" + str("{:n}".format(wait_time/1E-6))
    '''
    filename = date + '_lifetime_U2_' + str("{:.2f}".format(U2))+'_U1_' + str("{:.2f}".format(U1))  +'_Ex' + str("{:.2f}".format(Ex)) +'_Ey' + str("{:.2f}".format(Ey))+'_Ez' + str("{:.2f}".format(Ez)) + '_Prf' + str("{:.2f}".format(Prf)) +'dBm_Tload' + str("{:n}".format(pulse_width_ao/1E-6)) + 'us_Tej' \
    + str("{:n}".format(pulse_width_ej/1E-9)) + 'ns_wait' + str("{:n}".format(wait_time_corr/1E-6)) + 'us_reps'+ \
    str("{:.1e}".format(exp_cycles)) +'_trigger' + str("{:.1f}".format(trigger))+ 'CFD_off_+300V.txt'
    '''
    print (filename)
    ###############################
    # set up time tagger histogram and take data
    n_bin = int(time_window*(1000/bwidth)) # total time window 500 ns
    histo = Histogram(tagger, click_channel = 2, start_channel = 1, binwidth = bwidth, n_bins = n_bin)
#     histo = Histogram(tagger, click_channel = CFD_channels[0], start_channel = 1, binwidth = bwidth, n_bins = n_bin)
    tagger.sync()
    histo.clear()
    sleep(cycle_duration * exp_cycles)
    data = histo.getData()
    ###############################
    # save data
    roi_start = int(t_start * 1000/bwidth)  # start time of the roi
    roi_stop = int(t_stop *1000/bwidth)   # stop time of the roi
    x_data = np.arange(0, n_bin) * bwidth/1000 # x-axis, time in ns
#     file_data = np.array([x_data[0:1000], data[0:1000]])
    file_data = np.array([x_data, data])
    file_data = file_data.T
    np.savetxt(filename, file_data, delimiter=',')
    roi_count = np.sum(data[roi_start:roi_stop])
    print (wait_time*1E3, roi_count)
    variable = np.append(variable, wait_time)
    roi = np.append(roi, roi_count)

ROI_data = np.array([variable,roi])
ROI_data = ROI_data.T 
'''
    filename_roi = date + '_lifetime_ROI_U2_' + str("{:.2f}".format(U2))+'_U1_' + str("{:.2f}".format(U1)) +'_Ex' + str("{:.2f}".format(Ex)) +'_Ey' + str("{:.2f}".format(Ey)) +'_Ez' + str("{:.2f}".format(Ez))+'_Prf' + str("{:.2f}".format(Prf)) +'dBm_Tej' \
    + str("{:n}".format(pulse_width_ej/1E-9)) + 'ns_wait' + str("{:n}".format(wait_time_corr/1E-6)) + 'us_Tload' + str("{:n}".format(pulse_width_ao/1E-6)) + 'us_' + 'reps'+ \
    str("{:.1e}".format(exp_cycles)) + '_trigger' + str("{:.1f}".format(trigger))+ 'CFD_off_+300V.txt'
'''
filename_roi = date + "_ROI"
np.savetxt(filename_roi,ROI_data,delimiter=',')